In [3]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [4]:
df_train = pd.read_csv('data/covid19_data.csv')
df_test_pred = pd.read_csv('data/covid19_unlabelled_test.csv')

In [63]:
df_train.shape #(5531, 15)
df_train.tail()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang,label
5526,AFP,Argentina,MISLEADING,A video shows the Italian air force air show ...,2020/03/19,Argentina,NaN,NaN,NaN,https://factual.afp.com/el-video-del-espectacu...,poynter,El video del espectáculo aéreo de la fuerza aé...,Entradas similares circulan también en inglés....,NaN,es,f
5527,India Today,India,Mostly false,While coronavirus tests in other countries ar...,2020/03/25,India,NaN,NaN,NaN,https://www.indiatoday.in/fact-check/story/fac...,poynter,Fact Check: Are coronavirus tests most expensi...,"Speak Now, With more than 19,000 deaths and ov...",NaN,en,r
5528,TEMPO,Indonesia,Misleading,Vladimir Putin released 800 tigers andlions t...,2020/03/23,Indonesia,NaN,NaN,NaN,https://cekfakta.tempo.co/fakta/694/fakta-atau...,poynter,Tempo.co,Narasi yang menyebut Presiden Rusia Vladimir P...,NaN,id,f
5529,Factcheck.Vlaanderen,Belgium,misleading,The cure for coronavirus can come from Antwer...,2020/05/05,Belgium,NaN,NaN,NaN,https://factcheck.vlaanderen/factcheck/komt-de...,poynter,\n Factcheck: Komt de oplossing voor het co...,Een Antwerpse lama met de naam Winter is de ni...,NaN,nl,r
5530,Verificado,Mexico,Misleading,"Legislator from Nuevo León, Mexico says there...",2020/06/19,Mexico,NaN,NaN,NaN,https://verificado.com.mx/que-tan-certero-pued...,poynter,¿Qué tan certero puede ser el resultado de una...,En estado crítico atención hospitalaria para C...,NaN,es,f


In [ ]:
df_train

In [8]:
#function to classify the label in t(true) or f(false) or r(to revome)
def create_label(doc, list_false, list_true):
    
    if doc in list_false:
        return 'f'
    elif doc in list_true:
        return 't'
    else:
        return 'r'

#to select new data frame with the n_falses quantity of false observations
def create_balance_dataset(df,n_falses,label):
    df = df.copy()
    df = df[['title', label]]
    df_false = df[df[label]=='f'].sample(n=n_falses, random_state=111)
    df_true  = df[df[label]=='t']
    
    return pd.concat([df_false,df_true])       

    

In [11]:
list_false= ['False', 'FALSE','Misleading','MISLEADING']
list_true= ['True','News','Mostly True','MOSTLY TRUE','HALF TRUE','Correct Attribution','Half True','PARTLY TRUE',
            'half true','Correct','Partially true','HALF TRUTH','Partially correct', 'Mostly true','True but','Partly true',
           'NO EVIDENCE','no evidence','No Evidence','No evidence']

df_train['label'] = df_train['class'].apply(create_label, args=(list_false, list_true))

df_train['label'].value_counts(dropna=False)

f    4825
r     530
t     176
Name: label, dtype: int64

In [61]:
#trying oversample
df_train_tf = df_train[df_train['label'] != 'r']

sampling_strategy = 0.2

ros = RandomOverSampler(sampling_strategy=sampling_strategy,random_state=111)
X_res, y_res = ros.fit_resample(df_train_tf, df_train_tf['label'])

In [62]:
X_res['label'].value_counts(dropna=False)

f    4825
t     965
Name: label, dtype: int64

In [59]:
#trying undersample
sampling_strategy = 0.2

rus = RandomUnderSampler(sampling_strategy=sampling_strategy,random_state=111)
X_res_rus, y_res_rus = rus.fit_resample(df_train_tf, df_train_tf['label'])

In [60]:
X_res_rus['label'].value_counts(dropna=False)

f    880
t    176
Name: label, dtype: int64